In [0]:
checkpoint= "abfss://checkpoint@adlsexamplebiju.dfs.core.windows.net"  

In [0]:
def read_SilverTrafficTable():
    print("Reading the Silver Table Data")
    df_silverTraffic = (spark.readStream
                    .table("my_catalog.silver.silver_traffic")
                    )
    print(f'Reading y_catalog.silver.silver_traffic Success!')
    return df_silverTraffic

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import current_timestamp

In [0]:
def read_SilverRoadsTable():
    print("Reading the Silver Roads Table Data")
    df_silverRoads = (spark.readStream
                    .table("my_catalog.silver.silver_roads")
                    )
    print(f'Reading my_catalog.silver.silver_roads Success!')
    return df_silverRoads


In [0]:
from pyspark.sql.functions import col
from pyspark.sql.functions import col, format_number

In [0]:
def create_VehicleIntensity(df):
    print("Calculating the Road Density")
    print('Creating Vehicle Intensity column : ',end='')
    total=col('Motor_Vehicles_Count') / col('Link_length_km')
    df_veh = df.withColumn('Vehicle_Intensity',format_number(total,2)
               )
    print("Success")
    print('***************')
    return df_veh

In [0]:
def create_LoadTime(df):
    df_loadtime = df.withColumn("Load_Time",current_timestamp())
    print('Creating Load Time column : ',end='')
    return df_loadtime



In [0]:
def write_Traffic_GoldTable(StreamingDF):
    print('Writing the gold_traffic Data : ',end='') 

    write_gold_traffic = (StreamingDF.writeStream
                .format('delta')
                .option('checkpointLocation',checkpoint+ "/GoldTrafficLoad/Checkpt/")
                .outputMode('append')
                .queryName("GoldTrafficWriteStream")
                .trigger(availableNow=True)
                .toTable("my_catalog.gold.gold_traffic"))
    
    write_gold_traffic.awaitTermination()
    print(f'Writing my_catalog.gold.gold_traffic Success!')


In [0]:
def write_Roads_GoldTable(StreamingDF):
    print('Writing the gold_roads Data : ',end='') 

    write_gold_roads = (StreamingDF.writeStream
                .format('delta')
                .option('checkpointLocation',checkpoint+ "/GoldRoadsLoad/Checkpt/")
                .outputMode('append')
                .queryName("GoldRoadsWriteStream")
                .trigger(availableNow=True)
                .toTable("my_catalog.gold.gold_roads"))
    
    write_gold_roads.awaitTermination()
    print(f'Writing my_catalog.gold.gold_roads  Success!')

In [0]:
## Reading from Silver tables
df_SilverTraffic = read_SilverTrafficTable()
df_SilverRoads = read_SilverRoadsTable()
    
## Tranformations     
df_vehicle = create_VehicleIntensity(df_SilverTraffic)
df_FinalTraffic = create_LoadTime(df_vehicle)
df_FinalRoads = create_LoadTime(df_SilverRoads)


## Writing to gold tables    
write_Traffic_GoldTable(df_FinalTraffic)
write_Roads_GoldTable(df_FinalRoads)

In [0]:
%sql
select * from my_catalog.gold.gold_roads

In [0]:
%sql
select * from my_catalog.gold.gold_traffic

In [0]:
%sql
select count(1) from  my_catalog.gold.gold_traffic

In [0]:
%sql
select count(1) from  my_catalog.gold.gold_roads